## Analysis of Controller on Linear Simulation

This notebook generates the following figures in the paper

> Yeh, Christopher, et al. "Online learning for robust voltage control under uncertain grid topology." _arXiv preprint arXiv:2306.16674_ (2023).

- 2a
- 3a-e
- 5a

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

In [ ]:
from __future__ import annotations

from glob import glob
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm

from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('svg')

# hide top and right splines on plots
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

In [ ]:
plots_dir = 'plots/tsg_take2/'
os.makedirs(plots_dir, exist_ok=True)

def savefig(fig: plt.Figure, filename: str, **kwargs) -> None:
    path = os.path.join(plots_dir, filename)
    defaults = dict(dpi=300, pad_inches=0, bbox_inches='tight', facecolor='white')
    fig.savefig(path, **(defaults | kwargs))

In [ ]:
TIME_TICKS =  [   0, 2400, 4800,  7200,  9600, 12000, 14400]
TIME_LABELS = ['0h', '4h', '8h', '12h', '16h', '20h', '24h']

v_min, v_max = (11.4**2, 12.6**2)  # +/-5%, units kV^2
print(v_min, v_max)

y_min, y_max = 11.2, 12.8

In [ ]:
# Recreate Fig8 in Qu and Li (2020)
# - they count the substation as bus 1
# - we count the substation as bus 0
buses = [8, 18, 21, 30, 39, 45, 54]  # 0 = substation

In [ ]:
from network_utils import (
    create_56bus,
    create_RX_from_net,
    read_load_data)

net = create_56bus()
R, X = create_RX_from_net(net, noise=0)  # true R and X
p, qe = read_load_data()  # in MW and MVar
T, n = p.shape
print(f'T={T}, n={n}')

v_nom = 12**2  # nominal squared voltage magnitude, units kV^2
v_sub = v_nom  # fixed squared voltage magnitude at substation, units kV^2

vpars = qe @ X + p @ R + v_sub  # shape [T, n]

Vpar_min = np.min(vpars, axis=0)  # shape [n]
Vpar_max = np.max(vpars, axis=0)  # shape [n]
Vpar = (Vpar_min, Vpar_max)

In [ ]:
print('max-voltage node:', np.argmax(vpars.max(axis=0)))
print('min-voltage node:', np.argmin(vpars.min(axis=0)))

## Figure 2a: linear simulation, no controller

In [ ]:
def plot_2a(vpars: np.ndarray, save: bool = True):
    # plot linear sim no-control
    fig, ax = plt.subplots(figsize=(4, 3), dpi=200, tight_layout=True)

    ts = range(T)
    for i in np.asarray(buses) - 1:
        ax.plot(ts, np.sqrt(vpars[:, i]))

    ax.axhline(11.4, ls='--', color='black')
    ax.axhline(12.6, ls='--', color='black')
    ax.set(ylabel='Voltage (kV)', ylim=(11.0, 13.4))
    ax.set(xlabel='time $t$', xlim=(0, T),
           xticks=TIME_TICKS, xticklabels=TIME_LABELS)

    if save:
        savefig(fig, filename='linear_nocontrol.pdf')
        savefig(fig, filename='linear_nocontrol.png')
        savefig(fig, filename='linear_nocontrol.svg')

In [ ]:
plot_2a(vpars)

## Figures
1. voltage profile, robust controller with true X
2. voltage profile, robust controller+CBC with random X
3. voltage profile, robust controller+CBC with random X, known topo info for 14 buses
4. voltage profile, robust controller+CBC with random X, known topo+param info for 14 buses
5. error plot for $\|\hat{X}_t - X^*\|_\triangle$

In [ ]:
def load_pkl(path: str) -> dict:
    with open(path, 'rb') as f:
        return pickle.load(f)

In [ ]:
pkl_paths = {
    # ('known', None): 'out/CBCconst_20230809_234150.pkl',  # fixed X̂, fixed etahat
    ('known', None): 'out/CBCconst_δ20_η10_20230810_011115.pkl',  # fixed X̂, learned etahat
}
for seed in [8, 9, 10, 11]:
    pkl_paths |= {
        # default: δ=20
        ('unknown', seed): glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_2*.pkl')[0],
        ('topo-14', seed): glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_knowntopo14_2*.pkl')[0],
        ('lines-14', seed): glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],

        (r'η* known', seed): glob(f'out/CBCproj_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
        (r'δ=1', seed): glob(f'out/CBCproj_δ1_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
        (r'δ=20', seed): glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
        (r'δ=100', seed): glob(f'out/CBCproj_δ100_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
        (r'δ=500', seed): glob(f'out/CBCproj_δ500_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
    }

pkls = {}
for (name, seed), pkl_path in pkl_paths.items():
    pkl = load_pkl(pkl_path)
    pkls[(name, seed)] = pkl
    print(f'{name: <15} {str(seed): <4}', pkl.keys())

In [ ]:
from copy import deepcopy
mistakes = {
    'known': [], 'unknown': [], 'topo-14': [], 'lines-14': []
}
avg_viols = deepcopy(mistakes)
max_viols = deepcopy(mistakes)

figs_and_axs = {}
for seed in [8, 9, 10, 11]:
    figs_and_axs[seed] = plt.subplots(1, 3, figsize=(9, 3))

for key, pkl in pkls.items():
    vs = pkl['vs']
    assert vs.shape == (T, n)
    violates_max = (vs > v_max + 0.05)
    violates_min = (vs < v_min - 0.05)

    num_mistakes = (violates_max.any(axis=1) | violates_min.any(axis=1)).sum()
    num_bus_step_violations = violates_max.sum() + violates_min.sum()

    all_violations = np.concatenate([
        vs[violates_max] - v_max,
        v_min - vs[violates_min]
    ])

    avg_viol = np.mean(all_violations)
    max_viol = np.max(all_violations)

    seed = key[1]
    if seed in figs_and_axs:
        fig, axs = figs_and_axs[seed]
        if key[0] == 'unknown': ax = axs[0]
        elif key[0] == 'topo-14': ax = axs[1]
        elif key[0] == 'lines-14': ax = axs[2]
        ax.hist(all_violations, bins=np.arange(0, 8, 0.1))
        ax.set(xlabel='abs. violation', ylabel='count', title=str(key), yscale='log')

    num_updates = len(pkl['dists']['t']) - 1

    print(f'key: {key}, # updates: {num_updates}, '
          f'# mistakes: {num_mistakes}/{T}, '
          f'# bus-timestep violations: {num_bus_step_violations}, '
          f'avg viol: {avg_viol:.3g}, ',
          f'max viol: {max_viol:.3g}')

    mistakes[key[0]].append(num_mistakes)
    avg_viols[key[0]].append(avg_viol)
    max_viols[key[0]].append(max_viol)

for key in avg_viols:
    print(key)
    print(f'# mistakes: {np.mean(mistakes[key]):.1f} +/- {np.std(mistakes[key]):.1f}')
    print(f'avg violation: {np.mean(avg_viols[key]):.2f} +/- {np.std(avg_viols[key]):.2f}')
    print(f'max violation: {np.mean(max_viols[key]):.2f} +/- {np.std(max_viols[key]):.2f}')

In [ ]:
def check_consistency(data: dict):
    v = data['vs']
    qc = data['qcs']
    u = qc[1:] - qc[:-1]  # u[t] = u(t) = q^c(t+1) - q^c(t)
    Δv = v[1:] - v[:-1]  # Δv[t] = v(t+1) - v(t)
    count_inconsistent = []
    for t in sorted(data['params'].keys()):
        if data['config']['δ'] == 0:
            X̂ = data['params'][t]
            etahat = 8.65
        else:
            X̂, etahat = data['params'][t]
        vpar_hat = v[1:t+1] - qc[1:t+1] @ X̂
        w_hat = Δv[:t] - u[:t] @ X̂
        consistent = (
            (Vpar_min - 0.05 <= vpar_hat).all(axis=1)
            | (vpar_hat <= Vpar_max + 0.05).all(axis=1)
            | (np.max(np.abs(w_hat), axis=1) <= etahat)
        )
        num_inconsistent = t - consistent.sum()
        count_inconsistent.append(num_inconsistent)
    return count_inconsistent

In [ ]:
count_inconsistent = {}
for (name, seed), data in tqdm(pkls.items()):
    count_inconsistent[(name, seed)] = check_consistency(data)

In [ ]:
fig, axs = plt.subplots(1, 4, sharey=True, figsize=(15, 4))
for (name, seed), counts in count_inconsistent.items():
    if seed is None:
        continue
    ax = axs[seed % 8]
    ax.plot(counts, label=f'{name}')
    ax.legend(loc='upper left')
for ax, seed in zip(axs, [8, 9, 10, 11]):
    ax.set(xlabel='time $t$', title=f'seed {seed}')
    if seed == 8:
        ax.set(ylabel='# of inconsistent data points')
savefig(fig, filename='violations.png')

In [ ]:
def plot_pkl(name: str, data: dict, seed: int | None = None, plot_legend: bool = False) -> None:
    ts = range(T)
    fig, ax = plt.subplots(figsize=(4, 3), dpi=60, tight_layout=True)
    for i in np.array(buses) - 1:
        ax.plot(ts, np.sqrt(data['vs'][:, i]), label=f'{i+1}')

    ax.axhline(11.4, ls='--', color='black')
    ax.axhline(12.6, ls='--', color='black')
    ax.set(ylabel='Voltage (kV)', ylim=(y_min, y_max),
           yticks=[11.4, 11.7, 12, 12.3, 12.6])
    ax.set(xlabel='time $t$', xlim=(-50, T),
           xticks=TIME_TICKS, xticklabels=TIME_LABELS)

    filename = f'linear_{name}'
    if seed is not None:
        filename += f'_s{seed}'
    savefig(fig, filename=f'{filename}.pdf')
    savefig(fig, filename=f'{filename}.png')
    savefig(fig, filename=f'{filename}.svg')

    if plot_legend:
        leg = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='bus')
        fig.canvas.draw()
        bbox = leg.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        savefig(fig, 'linear_legend.pdf', bbox_inches=bbox)
        savefig(fig, 'linear_legend.png', bbox_inches=bbox)
        savefig(fig, 'linear_legend.svg', bbox_inches=bbox)

In [ ]:
def fig3abcd(seeds: list[int]) -> None:
    for seed in seeds:
        for i, name in enumerate(['unknown', 'topo-14', 'lines-14']):
            print(name, seed)
            data = pkls[(name, seed)]
            plot_pkl(name, data, seed, plot_legend=(i == 0))
    plot_pkl('known', pkls[('known', None)], None)


fig3abcd(seeds=[8, 9, 10, 11])

In [ ]:
def plot_error_and_etahat(pkls_dict: dict[str, dict], filename: str,
                          legend: str | None) -> tuple[plt.Figure, plt.Axes]:
    """
    Args
    - legend: one of [None, 'top', 'separate']
    """
    fig, ax = plt.subplots(figsize=(4, 3), dpi=60, tight_layout=True)
    axr = ax.twinx()
    axr.spines['right'].set_visible(True)

    for name, pkl in pkls_dict.items():
        print(name)
        dists = pkl['dists']
        ax.step(list(dists['t']) + [T], list(dists['X_true']) + [dists['X_true'][-1]],
                where='post', label=name)
        ax.scatter(0, dists['X_true'][0])
        if 'η' in dists:
            axr.step([0] + list(dists['t']) + [T],
                     [0] + list(dists['η']) + [dists['η'][-1]], ':',
                     where='post')
        else:
            axr.plot([0, T], [8.65, 8.65], ':')

    ax.set_ylabel(r'$||\hat{X}_t - X^\star||_\bigtriangleup$')
    axr.set_ylabel(r'$\hat\eta$')
    ax.set(xticks=TIME_TICKS, xticklabels=TIME_LABELS)
    ax.set(xlabel='time $t$', xlim=(-50, T))

    if legend == 'top':
        ax.legend(ncols=2, bbox_to_anchor=(0, 1), loc='lower left')

    savefig(fig, filename=f'{filename}.pdf')
    savefig(fig, filename=f'{filename}.png')
    savefig(fig, filename=f'{filename}.svg')

    if legend == 'separate':
        axr.set_ylabel('')
        axr.set_yticklabels([])
        leg = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        fig.canvas.draw()
        bbox = leg.get_tightbbox().transformed(fig.dpi_scale_trans.inverted())
        savefig(fig, f'{filename}_legend.pdf', bbox_inches=bbox)
        savefig(fig, f'{filename}_legend.png', bbox_inches=bbox)
        savefig(fig, f'{filename}_legend.svg', bbox_inches=bbox)

    return fig, ax

In [ ]:
def fig3e(seeds: list[int]) -> None:
    for seed in seeds:
        fig3e_pkls = {
            name: pkls[(name, seed)]
            for name in ['unknown', 'topo-14', 'lines-14']
        }
        fig3e_pkls['known'] = pkls[('known', None)]
        plot_error_and_etahat(
            fig3e_pkls, filename=f'linear_error_s{seed}', legend='top')


fig3e(seeds=[8, 9, 10, 11])

In [ ]:
def fig5a(seeds: list[int]) -> None:
    for i, seed in enumerate(seeds):
        pkls_by_delta = {
            name: pkls[(name, seed)]
            for name in ['η* known', 'δ=1', 'δ=20', 'δ=100', 'δ=500']
        }
        plot_error_and_etahat(
            pkls_by_delta, filename=f'linear_error_by_delta_s{seed}',
            legend='separate' if i == 0 else None)


fig5a(seeds=[8, 9, 10, 11])

## Detecting network change

In [ ]:
seed = 8
pkl_paths = {
    # default: δ=20
    'unknown': glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_topochange_2*.pkl')[0],
    'topo-14': glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_knowntopo14_topochange_2*.pkl')[0],
    'lines-14': glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_topochange_2*.pkl')[0],
}

pkls = {}
for name, pkl_path in pkl_paths.items():
    pkl = load_pkl(pkl_path)
    pkls[name] = pkl
    print(f'{name: <15}', pkl.keys())

    pkl['dists1'] = pkl['dists']
    pkl['dists'] = {
        'X_true': np.concatenate([pkl['dists1']['X_true'], pkl['dists2']['X_true']]),
        'η': np.concatenate([pkl['dists1']['η'], pkl['dists2']['η']]),
        't': np.concatenate([pkl['dists1']['t'], np.array(pkl['dists2']['t']) + int(T/2)]),
    }

In [ ]:
fig, ax = plot_error_and_etahat(
    pkls, filename=f'linear_detectchange_s{seed}', legend='top')

## Figures (Alternative)

Only plot voltages for two buses: 18 and 30 (where bus 0 is substation)

In [ ]:
def plot_bus(bus: int, pkls: dict, plot_legend: bool = False) -> None:
    ts = range(T)
    fig, ax = plt.subplots(figsize=(4, 3), dpi=200, tight_layout=True)

    for name, data in pkls.items():
        ax.plot(ts, np.sqrt(data['vs'][:, bus-1]), label=name)

    ax.axhline(11.4, ls='--', color='black')
    ax.axhline(12.6, ls='--', color='black')
    ax.set(ylabel='Voltage (kV)', ylim=(y_min, y_max),
           yticks=[11.4, 11.7, 12, 12.3, 12.6])
    ax.set(xlabel='time $t$', xlim=(-50, T),
           xticks=TIME_TICKS, xticklabels=TIME_LABELS)

    savefig(fig, filename=f'linear_bus{bus}.pdf')
    savefig(fig, filename=f'linear_bus{bus}.png')
    savefig(fig, filename=f'linear_bus{bus}.svg')

    if plot_legend:
        leg = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        fig.canvas.draw()
        bbox = leg.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('plots/tsg/linear_legend2.pdf', dpi=300, bbox_inches=bbox, facecolor='white')
        fig.savefig('plots/tsg/linear_legend2.png', dpi=300, bbox_inches=bbox, facecolor='white')

In [ ]:
for i, bus in enumerate([18, 30]):
    plot_bus(bus, pkls_by_seed[10], plot_legend=(i==0))